# Detecton of sugarbeet plants with YOLO11n

### getting stated
this notebook guides you in several stef to create and apply a detector for Sugarbeets in images. 
You will need the dataset which was created from Fendt GmbH.

important steps in this Notebook
- prepare the given dataset for YOLO
- split the data and move it to seperate folders
- train the model
- apply the model 

### 1. Make txt-files from json-files

Note: copy the sugarbeet dataset in workspace. Directory should be in the original form:

```
|datensatz_sugarbeet/
|   ├── annotations/
|   └── img/
```

skript creates folder convertedSugarbeetDataset and convertes json files to TXT files according to the requirements of YOLO

In [5]:
import os
import json

annotations_dir = "C:/Users/Philip/Documents/Fendt_KI_Challenge/yolo_training/labels/train"  # directory of annotation data stored in JSON-files 
output_dir = "C:/Users/Philip/Documents/Fendt_KI_Challenge/yolo_training/labels/yolo_txt"  # directory for YOLO-TXT-files

# define classes 
CLASSES = {
    "sugarbeet": 0,
    #"weed": 1,
    #"rock": 2,
}

# create output directory if doesnt exist
os.makedirs(output_dir, exist_ok=True)


def convert_to_yolo_format(img_width, img_height, bbox):
    """convert bounding box coordinates in YOLO format.
    
    inputparameters:    img_width           int                                        width of image
                        img_height          int                                        height of image
                        bbox                list: [[x_min, y_min],[x_max, y_max]]      absolute coordinates according to image width and height.
                        
    return:             x_center            int                                        relative x-coordinate of bounding box
                        y_center            int                                        relative y-coordinate of bounding box
                        width               int                                        with of bounding box
                        height              int                                        height  of bounding box"""
    
    x_min, y_min = bbox[0]
    x_max, y_max = bbox[1]

    # calculate YOLO-coordinates (relativ to image size)
    x_center = ((x_min + x_max) / 2) / img_width
    y_center = ((y_min + y_max) / 2) / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height

    return x_center, y_center, width, height


def process_json_file(json_file):
    """read JSON file and convert bounding box coordinates in YOLO format.

    inputparameters:       json_file           string          path to json file
    
    returns:               yolo_lines          list            list of strings with content of classes and boundingboxes in YOLO-Format"""

    with open(json_file, "r") as f:
        data = json.load(f)

    img_width = data["imgWidth"]
    img_height = data["imgHeight"]
    objects = data["objects"]

    yolo_lines = []
    for obj in objects:
        label = obj["label"]
        if label in CLASSES:
            class_id = CLASSES[label]
            bbox = obj["bbox"]
            yolo_bbox = convert_to_yolo_format(img_width, img_height, bbox)
            yolo_line = f"{class_id} {' '.join(map(str, yolo_bbox))}"
            yolo_lines.append(yolo_line)
    
    return yolo_lines


def main():
    '''iterate over all json files an create TXT Files with data for classes and bounding boxes according to the requirements of YOLO'''

    for json_file in os.listdir(annotations_dir):
        if json_file.endswith(".json"):
            json_path = os.path.join(annotations_dir, json_file)
            yolo_lines = process_json_file(json_path)

            # write data in TXT files
            txt_file = os.path.splitext(json_file)[0] + ".txt"
            txt_path = os.path.join(output_dir, txt_file)
            with open(txt_path, "w") as f:
                f.write("\n".join(yolo_lines))

            print(f"Konvertiert: {json_file} -> {txt_file}")

if __name__ == "__main__":
    main()


Konvertiert: sugarbeet_bbch12_000002.json -> sugarbeet_bbch12_000002.txt
Konvertiert: sugarbeet_bbch12_000003.json -> sugarbeet_bbch12_000003.txt
Konvertiert: sugarbeet_bbch12_000004.json -> sugarbeet_bbch12_000004.txt
Konvertiert: sugarbeet_bbch12_000005.json -> sugarbeet_bbch12_000005.txt
Konvertiert: sugarbeet_bbch12_000006.json -> sugarbeet_bbch12_000006.txt
Konvertiert: sugarbeet_bbch12_000007.json -> sugarbeet_bbch12_000007.txt
Konvertiert: sugarbeet_bbch12_000008.json -> sugarbeet_bbch12_000008.txt
Konvertiert: sugarbeet_bbch12_000009.json -> sugarbeet_bbch12_000009.txt
Konvertiert: sugarbeet_bbch12_000010.json -> sugarbeet_bbch12_000010.txt
Konvertiert: sugarbeet_bbch12_000011.json -> sugarbeet_bbch12_000011.txt
Konvertiert: sugarbeet_bbch12_000013.json -> sugarbeet_bbch12_000013.txt
Konvertiert: sugarbeet_bbch12_000014.json -> sugarbeet_bbch12_000014.txt
Konvertiert: sugarbeet_bbch12_000015.json -> sugarbeet_bbch12_000015.txt
Konvertiert: sugarbeet_bbch12_000016.json -> sugarb

### 

### 2. Move data in seperate folders

- script splits data in 80% train und 20% validation 


In [6]:
import os
import shutil
import random

# define directorys
image_dir = "C:/Users/Philip/Documents/Fendt_KI_Challenge/yolo_training/images/train"
label_dir = "C:/Users/Philip/Documents/Fendt_KI_Challenge/yolo_training/labels/yolo_txt"
output_dir = "C:/Users/Philip/Documents/Fendt_KI_Challenge/yolo_training/output_train"

# build paths 
train_img_dir = os.path.join(output_dir, "images/train")
val_img_dir = os.path.join(output_dir, "images/val")
train_label_dir = os.path.join(output_dir, "labels/train")
val_label_dir = os.path.join(output_dir, "labels/val")

# create folder for splited data
os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

split_ratio = 0.8  # 80% Training, 20% Validation

def split_data(image_dir, label_dir, train_img_dir, val_img_dir, train_label_dir, val_label_dir, split_ratio):
    '''
    Splits data randomly and move them in recently created directorys

    inputparameters:    severals paths (string) of directorys: image_dir, label_dir, train_img_dir, val_img_dir, train_label_dir, val_label_dir,
                        split_ratio:            int             defines ratio of All data and train data
    '''
    # list of all images and labels of associated labels
    images = [f for f in os.listdir(image_dir) if f.endswith((".png", ".jpg", ".jpeg"))]
    random.shuffle(images)  # Zufällige Reihenfolge

    # calculate number of train data
    split_index = int(len(images) * split_ratio)

    # split in train and validation data
    train_images = images[:split_index]
    val_images = images[split_index:]

    # move files
    for image_set, img_dir, label_dir_target in [
        (train_images, train_img_dir, train_label_dir),
        (val_images, val_img_dir, val_label_dir),
    ]:
        for image_file in image_set:
            # Bild verschieben
            src_img_path = os.path.join(image_dir, image_file)
            dst_img_path = os.path.join(img_dir, image_file)
            shutil.copy(src_img_path, dst_img_path)

            # Zugehörige Label-Datei verschieben
            label_file = os.path.splitext(image_file)[0] + ".txt"
            src_label_path = os.path.join(label_dir, label_file)
            if os.path.exists(src_label_path):  # Prüfe, ob Label existiert
                dst_label_path = os.path.join(label_dir_target, label_file)
                shutil.copy(src_label_path, dst_label_path)

    print("splitting completed!")
    print(f"train data: {len(train_images)}")
    print(f"validation data: {len(val_images)}")

if __name__ == "__main__":
    split_data(image_dir, label_dir, train_img_dir, val_img_dir, train_label_dir, val_label_dir, split_ratio)


splitting completed!
train data: 218
validation data: 55


### 3. Start training

- Note: change the paths in sugarbeet_dataset.yaml to your absolute pahts to the train and validation directorys
- 100 Epochs ~ 3 hours
- skript saves the best model in runs\detect\train2\weights

In [3]:
from ultralytics import YOLO
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device: ", device)

# load pretrained model
#https://docs.ultralytics.com/de/modes/train/#train-settings
#https://pytorch.org/vision/master/models/faster_rcnn.html
model = YOLO("yolo11l.pt") #x probieren
#1280 model Bryan, inference Zeit nehmen
# Train the model with your data
results = model.train(data="sugarbeet_dataset.yaml", epochs=150, imgsz=640, device=0, augment=True) # augment=False, img size anpassen, dropout, val = True (default), pretrained = True (default)

Device:  cuda
New https://pypi.org/project/ultralytics/8.3.62 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.61  Python-3.12.4 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3080, 12288MiB)
engine\trainer: task=detect, mode=train, model=yolo11l.pt, data=sugarbeet_dataset.yaml, epochs=150, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train25, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, sho

train: Scanning C:\Users\Philip\Documents\Fendt_KI_Challenge\yolo_training\output_train\labels\train.cache... 218 images, 0 backgrounds, 0 corrupt: 100%|██████████| 218/218 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\Philip\Documents\Fendt_KI_Challenge\yolo_training\output_train\labels\val.cache... 55 images, 0 backgrounds, 0 corrupt: 100%|██████████| 55/55 [00:00<?, ?it/s]


Plotting labels to runs\detect\train25\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train25
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    150/150      11.1G     0.5541     0.2809     0.8542        340        640: 100%|██████████| 14/14 [00:03<00:00,  3.62it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.46it/s]

                   all         55       2373      0.964      0.942      0.977      0.805



150 epochs completed in 0.805 hours.
Optimizer stripped from runs\detect\train25\weights\last.pt, 51.2MB
Optimizer stripped from runs\detect\train25\weights\best.pt, 51.2MB

Validating runs\detect\train25\weights\best.pt...
Ultralytics 8.3.61  Python-3.12.4 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3080, 12288MiB)
YOLO11l summary (fused): 464 layers, 25,280,083 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.11it/s]


                   all         55       2373       0.94      0.931      0.974      0.792
Speed: 0.1ms preprocess, 21.7ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\train25


### 4. Detect sugarbeets with best model

- replace source with image or directory of images
- model saves copy of pictures with bounding boxes in runs\detect\predict
- prints predected bounding boxes
- plot image with sugarbeet predictions

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load the trained model
model = YOLO("runs/detect/train22/weights/best.pt")
img_path = "C:/Users/Philip/Documents/Fendt_KI_Challenge/Test/images"

results = model.predict(source="C:/Users/Philip/Documents/Fendt_KI_Challenge/Test/images/sugarbeet_bbch12_000138.png", save=False, imgsz=640)

# Access predictions
for result in results:
    boxes = result.boxes.xyxy.cpu().numpy()  # all Bounding-Box-coordinates
    classes = result.boxes.cls.cpu().numpy()  # class-IDs
    sugarbeet_boxes = boxes[classes == 0] # all bounding boxes of sugarbeets
    print("Sugarbeet Boxes:", sugarbeet_boxes)

# plot image with predicted boundingboxes
image = mpimg.imread('runs/detect/predict6/sugarbeet_bbch12_000138.jpg')
plt.imshow(image)
plt.axis('off')
plt.show()


<Figure size 1200x800 with 1 Axes>

<Figure size 1200x800 with 1 Axes>

<Figure size 1200x800 with 1 Axes>

<Figure size 1200x800 with 1 Axes>

<Figure size 1200x800 with 1 Axes>

...

<Figure size 1200x800 with 1 Axes>

<Figure size 1200x800 with 1 Axes>

<Figure size 1200x800 with 1 Axes>

<Figure size 1200x800 with 1 Axes>

<Figure size 1200x800 with 1 Axes>